# Auxiliary layers - DEV

Here we create 

* a raster that is empty - is this useful ?
* a raster with the distance to the raster border - used for selecting pixels in a multi-tile project
* a raster with the distance to the polygon border - useful for selecting clean training samples

**TODO**: Create a Snakemake task.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# - - - - - - - - - - - - - - - - - - - - 
# DEFAULT IMPORTS - IN ALL NOTEBOKS
from src import configs

prjconf = configs.ProjectConfigParser()

# - - - - - - - - - - - - - - - - - - - - 
# NOTEBOOK SPECIFIC IMPORTS
import numpy as np
from pathlib import Path
import pandas as pd
import rasterio
from tqdm import tqdm

from eobox.raster import cube
from eobox.raster import gdalutils
from eobox.raster.rasterprocessing import create_distance_to_raster_border
from eobox.vector import calc_distance_to_border


def create_empty_raster(tile, overwrite=False):
    """Create an empty raster in the *timeless* folder from first *__QA.tif* match in 'Interim' 'hls'."""
    
    path_empty_raster = prjconf.get_path("Interim", "timeless_empty_raster", tile=tile)
    path_empty_raster.parent.mkdir(exist_ok=True, parents=True)
    
    if not path_empty_raster.exists() or overwrite:
        # get the first QA layer as a raster template. that could be anything
        for pth in (prjconf.get_path("Interim", "hls") / tile).rglob("**/*__QA.tif"):
            path_template_raster = pth
            break
        # create an empty raster 
        path_empty_raster.parent.mkdir(exist_ok=True, parents=True)
        with rasterio.open(path_template_raster) as src:
            meta = src.meta
            meta.update(dtype='uint8')
            arr = (src.read() * 0).astype('uint8')
            with rasterio.open(path_empty_raster, 'w', **meta) as dst:
                dst.write(arr)
    return path_empty_raster

tilenames = prjconf.get("Params", "tiles").split(" ")
tilenames



['32UNU', '32UPU', '32UQU', '33UUP']

## Inputs

### Parameters

In [2]:
path_ref_vector_all_tiles = {}
for tile in tilenames:
    print(tile)
    
    path_ref_vector = prjconf.get_path("Interim", "clc_subset5", tile=tile)
    
    # create an empty raster to be used as template
    create_empty_raster(tile)
    
    if False: # NOT USED ANYMORE: THIS IS NOW DONE DURING EXTRACTION !!!
        # create distance to raster border to be used in a multi-tile project
        # after extraction, this auxiliar layer togehter with the coordinates helps to decide
        # which pixel to take, given that they have been extracted multiple times in overlapping areas
        # reasoning: we want the pixels which are most distance from the raster border
        path_empty_raster = prjconf.get_path("Interim", "timeless_empty_raster", tile=tile)
        path_dist2rb = prjconf.get_path("Interim", "timeless_dist2rb", tile=tile)

        create_distance_to_raster_border(src_raster = path_empty_raster, # could be any raster
                                         dst_raster = path_dist2rb,
                                         maxdist=None, # None means we calculate distances for all pixels
                                         overwrite=False)

        # create distance to polygon border layer useful for filtering training samples / select purer inner polygon pixels
        path_dist2pb = prjconf.get_path("Interim", "timeless_dist2pb", tile=tile)
        path_dist2pb = str(path_dist2pb).replace("VECTORNAME", path_ref_vector.stem)

        calc_distance_to_border(polygons=path_ref_vector,
                                template_raster=path_empty_raster,
                                dst_raster=path_dist2pb,
                                overwrite=False,
                                keep_interim_files=False)
    
    # CREATED FILES
    print(path_empty_raster)
    print(path_dist2rb)
    print(path_dist2pb)

32UNU
/home/ben/Devel/Projects/classify-hls/data/interim/timeless/32UNU/32UNU_empty_raster.tif
/home/ben/Devel/Projects/classify-hls/data/interim/timeless/32UNU/32UNU_dist2rb.tif
/home/ben/Devel/Projects/classify-hls/data/interim/timeless/32UNU/32UNU_dist2pb_clc2018_32UNU_subset_500000.tif
32UPU
/home/ben/Devel/Projects/classify-hls/data/interim/timeless/32UPU/32UPU_empty_raster.tif
/home/ben/Devel/Projects/classify-hls/data/interim/timeless/32UPU/32UPU_dist2rb.tif
/home/ben/Devel/Projects/classify-hls/data/interim/timeless/32UPU/32UPU_dist2pb_clc2018_32UPU_subset_500000.tif
32UQU
/home/ben/Devel/Projects/classify-hls/data/interim/timeless/32UQU/32UQU_empty_raster.tif
/home/ben/Devel/Projects/classify-hls/data/interim/timeless/32UQU/32UQU_dist2rb.tif
/home/ben/Devel/Projects/classify-hls/data/interim/timeless/32UQU/32UQU_dist2pb_clc2018_32UQU_subset_500000.tif
33UUP
/home/ben/Devel/Projects/classify-hls/data/interim/timeless/33UUP/33UUP_empty_raster.tif
/home/ben/Devel/Projects/classif